In [1]:
!pip install influxdb_client influxdb
!pip install kafka-python
!pip install pysad
!pip install combo

In [1]:
from kafka import KafkaConsumer
import pandas as pd
from pyspark.sql import SparkSession
import ast
import json
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
import numpy as np
from pysad.transform.preprocessing import InstanceUnitNormScaler

## Online K-MEAN

In [2]:
class KMean:
    def __init__(self, k, d):
        """
        Does an online k-means update on a single data point.
        point - a 1 x d array
        k - integer > 1 - number of clusters
        cluster_means - a k x d array of the means of each cluster
        cluster_counts - a 1 x k array of the number of points in each cluster
        """
        self.k = k
        self.d = d
        self.point = None
        self.current_cluster = None
        self.cluster_means = np.random.normal(size=[self.k, self.d], loc=0.5, scale=1)
        self.cluster_counts = np.zeros((self.k,self.d))
        
    def add_point(self, point):
        self.point = point
    
    def update(self):
        cluster_distances = np.zeros(self.k)
        for cluster in range(self.k):
            cluster_distances[cluster] = sum(np.sqrt((self.point - self.cluster_means[cluster])**2))
        self.current_cluster = np.argmin(cluster_distances)
        self.cluster_counts[self.current_cluster] += 1
        self.cluster_means[self.current_cluster] += 1.0/self.cluster_counts[self.current_cluster]*(self.point - self.cluster_means[self.current_cluster])
    
    def query(self):
        return self.current_cluster
    
    def __call__(self, point):
        self.add_point(point)
        self.update()
        return self.query()
    

In [3]:
from sklearn import cluster

class BRICH:
    def __init__(self, n_clusters=2, threshold=0.2):
        """
        BIRCH clustering algorithm
        point - 1 x d array
        n_clusters - number of clusters
        threshold - The radius of the subcluster 
        """
        self.n_clusters = n_clusters
        self.threshold = threshold
        self.point = None
        self.current_cluster = None
        self.brich = cluster.Birch(n_clusters=self.n_clusters,  threshold=self.threshold)
        
    def add_point(self, point):
        self.point = point
    
    def update(self):
        self.brich.partial_fit(np.array(self.point).reshape(1,-1))
    
    def query(self):
        return self.brich.labels_[0].tolist()
    
    def __call__(self, point):
        self.add_point(point)
        self.update()
        return self.query()
    

## Sending data to InfluxDB

In [4]:
token = "qS9oBP6E51R0kduMfOB0e2zWkwGMuY6w09d00mCCmLMSIrp-N5bylJjoz4slo-7nk9yvdaKoKg9ii7KZ5ItyCg=="
org = "ost"
bucket = "clustering"
url = "http://influxdb:8086"

In [5]:
spark = SparkSession.builder.appName("swat").getOrCreate()
consumer = KafkaConsumer('swat_elte',bootstrap_servers=['kafka-server:9092'],
                         group_id=None,
                         # auto_offset_reset='earliest', 
                         api_version=(0,10,1))

In [6]:
kmean = None
brich = BRICH()
with influxdb_client.InfluxDBClient(url=url, token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for message in consumer:
        message = message.value
        if len(message) < 10:
            break
        try:
            point = json.loads(message)
            point_values = []
            for key in point:
                p = influxdb_client.Point("point").field(key, point[key])
                write_api.write(bucket, org, p)
                point_values.append(point[key])
            if kmean is None:
                kmean = KMean(2, len(point_values))
            print(point_values)
            kmean_cluster = kmean(point_values)
            brich_cluster = brich(point_values)
            p = influxdb_client.Point("point").field("kmean_cluster", kmean_cluster)
            write_api.write(bucket, org, p)
            p = influxdb_client.Point("point").field("brich_cluster", brich_cluster)
            write_api.write(bucket, org, p)
        except NotImplementedError:
            pass

[0.807990253]
[0.8086305]
[0.80927074]


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(


[0.80927074]
[0.80927074]
[0.80927074]
[0.80927074]
[0.8069659]
[0.8069659]
[0.8069659]
[0.808758557]


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea

[0.808758557]
[0.807350039]
[0.8056854]
[0.8056854]
[0.8056854]
[0.8077342]


...

[0.7951853]
[0.7965939]
[0.7965939]
[0.7965939]
[0.7973622]
[0.7973622]
[0.7973622]
[0.7997951]
[0.7997951]
[0.7997951]
[0.7997951]
[0.7997951]
[0.7997951]
[0.7997951]


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea

[0.7997951]


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_birch.py:752: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (2). Decrease the threshold.
  warnings.warn(


KeyboardInterrupt: 